# Adjusting currencies 

## Extracting the prices from Barcelona json file

In [2]:
import json
import pandas as pd
import os

#Open the madrid coworkings csv
path = '/workspaces/Coworking/src/results/Barcelona/Barcelona_coworking_spaces.json'

with open(path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.json_normalize(data)

df_barcelona_price= df[['price']]

In [3]:
import re

# Function to extract the price
def extract_price(text):
    if pd.isna(text):  # Handle missing values
        return None
    match = re.search(r'(\d+)', text)  # Extract the first number
    return int(match.group(1)) if match else None  # Convert to int if found

# Apply the function to the 'price' column
df_barcelona_price['price_cleaned'] = df_barcelona_price['price'].apply(extract_price)

# Display the results
print(df_barcelona_price[['price', 'price_cleaned']])

                                  price  price_cleaned
0   Private Office\n from € 500\n/month            500
1   Private Office\n from € 305\n/month            305
2   Private Office\n from € 245\n/month            245
3  Private Office\n from € 1214\n/month           1214
4   Private Office\n from € 249\n/month            249
5   Private Office\n from € 350\n/month            350
6                         1\n2\n3\n4\n5              1
7   Private Office\n from € 249\n/month            249
8   Private Office\n from € 345\n/month            345
9   Private Office\n from € 155\n/month            155


/tmp/ipykernel_26605/1902064055.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_barcelona_price['price_cleaned'] = df_barcelona_price['price'].apply(extract_price)


## Extract the price from Madrid json file.

In [4]:
import json
import pandas as pd
import os

#Open the madrid coworkings csv
path = '/workspaces/Coworking/src/results/Madrid/Madrid_coworking_spaces.json'

with open(path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.json_normalize(data)

df_madrid_price= df[['price']]

In [5]:
import re

# Function to extract the price
def extract_price(text):
    if pd.isna(text):  # Handle missing values
        return None
    match = re.search(r'(\d+)', text)  # Extract the first number
    return int(match.group(1)) if match else None  # Convert to int if found

# Apply the function to the 'price' column
df_madrid_price['price_cleaned'] = df_madrid_price['price'].apply(extract_price)

# Display the results
print(df_madrid_price[['price', 'price_cleaned']])

                                   price  price_cleaned
0    Private Office\n from € 490\n/month          490.0
1    Private Office\n from € 189\n/month          189.0
2    Private Office\n from € 315\n/month          315.0
3    Private Office\n from € 980\n/month          980.0
4    Private Office\n from € 225\n/month          225.0
..                                   ...            ...
180    Private Office \nPrice on request            NaN
181    Private Office \nPrice on request            NaN
182    Private Office \nPrice on request            NaN
183    Private Office \nPrice on request            NaN
184                        1\n2\n3\n4\n5            1.0

[185 rows x 2 columns]


/tmp/ipykernel_26605/871941500.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_madrid_price['price_cleaned'] = df_madrid_price['price'].apply(extract_price)


## Merge both prices dataframes.

In [6]:
import pandas as pd

# Concatenate both dataframes
df_euro = pd.concat([df_madrid_price, df_barcelona_price], ignore_index=True)

# Optionally, you can reset the index if needed
df_euro.reset_index(drop=True, inplace=True)

df_euro.head()

,price,price_cleaned
0,Private Office\n from € 490\n/month,490.0
1,Private Office\n from € 189\n/month,189.0
2,Private Office\n from € 315\n/month,315.0
3,Private Office\n from € 980\n/month,980.0
4,Private Office\n from € 225\n/month,225.0


In [7]:
df_euro.drop(columns='price', inplace=True)

df_euro.head()

,price_cleaned
0,490.0
1,189.0
2,315.0
3,980.0
4,225.0


### Transform NaN values with the median.

In [8]:
df_euro.isna().sum()

price_cleaned    14
dtype: int64

In [9]:
median_price = df_euro['price_cleaned'].median()
df_euro['price_cleaned'].fillna(median_price, inplace=True)

/tmp/ipykernel_26605/1063668431.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_euro['price_cleaned'].fillna(median_price, inplace=True)


In [10]:
df_euro.isna().sum()

price_cleaned    0
dtype: int64

## Convert Euro in USD and normalize

### Using ExchangeRate-API

In [11]:
import pandas as pd
import requests

# Define your API endpoint and your API key
api_key = "7352f0026f58a51e76be2471"  # Replace with your API key
api_url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/EUR"

# Fetch the EUR to USD conversion rate from the API
def get_eur_to_usd_rate():
    response = requests.get(api_url)
    data = response.json()
    
    # Check if the response is valid
    if data['result'] == 'success':
        return data['conversion_rates']['USD']
    else:
        raise Exception("Error fetching exchange rate from API")

# Convert EUR to USD
def convert_eur_to_usd(eur_price, conversion_rate):
    return eur_price * conversion_rate

# Fetch the conversion rate (EUR to USD)
conversion_rate = get_eur_to_usd_rate()

# Apply the conversion to the 'price' column
df_euro['price_usd'] = df_euro['price_cleaned'].apply(lambda x: convert_eur_to_usd(x, conversion_rate))

# Print the updated DataFrame
print(df_euro)


     price_cleaned  price_usd
0            490.0   530.3760
1            189.0   204.5736
2            315.0   340.9560
3            980.0  1060.7520
4            225.0   243.5400
..             ...        ...
190          350.0   378.8400
191            1.0     1.0824
192          249.0   269.5176
193          345.0   373.4280
194          155.0   167.7720

[195 rows x 2 columns]


In [12]:
df_euro.drop(columns='price_cleaned', inplace=True)

df_euro.head()

,price_usd
0,530.3760
1,204.5736
2,340.9560
3,1060.7520
4,243.5400


### The PPP factor of Spain is 1(LCU/Euro)(Local Currency Unit)

## Convert Yens in USD and use the PPP factor.

In [13]:
import json
import pandas as pd
import os

#Open the madrid coworkings csv
path = '/workspaces/Coworking/src/results/Tokyo/tokyo_coworking_spaces.json'

with open(path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.json_normalize(data)

df_tokyo_price= df[['price']]

In [14]:
import re

df_tokyo_price['price_cleaned'] = df_tokyo_price['price'].apply(extract_price)

# Display the results
print(df_tokyo_price[['price', 'price_cleaned']])

                                   price  price_cleaned
0  Private Office\n from ¥ 57500\n/month          57500
1                          1\n2\n3\n4\n5              1
2  Private Office\n from ¥ 56000\n/month          56000
3  Private Office\n from ¥ 43000\n/month          43000
4  Private Office\n from ¥ 71500\n/month          71500
5  Private Office\n from ¥ 71900\n/month          71900
6  Private Office\n from ¥ 48000\n/month          48000
7  Private Office\n from ¥ 58500\n/month          58500
8  Private Office\n from ¥ 69900\n/month          69900
9  Private Office\n from ¥ 55500\n/month          55500


/tmp/ipykernel_26605/3197838495.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tokyo_price['price_cleaned'] = df_tokyo_price['price'].apply(extract_price)


In [15]:
df_tokyo_price.isna().sum()

price            0
price_cleaned    0
dtype: int64

In [16]:
df_tokyo_price.drop(columns='price', inplace=True)

/tmp/ipykernel_26605/1139496757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tokyo_price.drop(columns='price', inplace=True)


In [17]:
import pandas as pd
import requests

# Define your API endpoint and your API key
api_key = "7352f0026f58a51e76be2471"  # Replace with your API key
api_url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/EUR"

# Fetch the EUR to USD conversion rate from the API
def get_eur_to_usd_rate():
    response = requests.get(api_url)
    data = response.json()
    
    # Check if the response is valid
    if data['result'] == 'success':
        return data['conversion_rates']['USD']
    else:
        raise Exception("Error fetching exchange rate from API")

# Convert EUR to USD
def convert_eur_to_usd(eur_price, conversion_rate):
    return eur_price * conversion_rate

# Fetch the conversion rate (EUR to USD)
conversion_rate = get_eur_to_usd_rate()

# Apply the conversion to the 'price' column
df_tokyo_price['price_usd'] = df_tokyo_price['price_cleaned'].apply(lambda x: convert_eur_to_usd(x, conversion_rate))

# Print the updated DataFrame
print(df_tokyo_price)


   price_cleaned   price_usd
0          57500  62238.0000
1              1      1.0824
2          56000  60614.4000
3          43000  46543.2000
4          71500  77391.6000
5          71900  77824.5600
6          48000  51955.2000
7          58500  63320.4000
8          69900  75659.7600
9          55500  60073.2000


/tmp/ipykernel_26605/4015434545.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tokyo_price['price_usd'] = df_tokyo_price['price_cleaned'].apply(lambda x: convert_eur_to_usd(x, conversion_rate))


In [18]:
df_tokyo_price.drop(columns='price_cleaned', inplace=True)

/tmp/ipykernel_26605/1048160013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tokyo_price.drop(columns='price_cleaned', inplace=True)


### The PPP of Japan is 95(LSU/USD)

In [19]:
df_tokyo_price['price_usd'] = df_tokyo_price['price_usd']*95

df_tokyo_price.head()

/tmp/ipykernel_26605/1995806028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tokyo_price['price_usd'] = df_tokyo_price['price_usd']*95


,price_usd
0,5912610.000
1,102.828
2,5758368.000
3,4421604.000
4,7352202.000


## Convert Real into USD and use the PPP factor.

In [20]:
import json
import pandas as pd
import os

#Open Sao Paulo json
path = '/workspaces/Coworking/src/results/Sao Paulo/Sao_Paulo_coworking_spaces.json'

with open(path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.json_normalize(data)

df_brazil_price= df[['price']]

In [21]:
import re

# Function to extract the price
def extract_price(text):
    if pd.isna(text):  # Handle missing values
        return None
    match = re.search(r'(\d+)', text)  # Extract the first number
    return int(match.group(1)) if match else None  # Convert to int if found

# Apply the function to the 'price' column
df_brazil_price['price_cleaned'] = df_brazil_price['price'].apply(extract_price)

# Display the results
print(df_brazil_price[['price', 'price_cleaned']])

                                   price  price_cleaned
0  Private Office\n from R$ 1815\n/month           1815
1                          1\n2\n3\n4\n5              1
2  Private Office\n from R$ 1389\n/month           1389
3  Private Office\n from R$ 1565\n/month           1565
4  Private Office\n from R$ 1015\n/month           1015
5  Private Office\n from R$ 1385\n/month           1385
6  Private Office\n from R$ 2209\n/month           2209
7  Private Office\n from R$ 1899\n/month           1899
8  Private Office\n from R$ 1059\n/month           1059
9  Private Office\n from R$ 1299\n/month           1299


/tmp/ipykernel_26605/1699147551.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brazil_price['price_cleaned'] = df_brazil_price['price'].apply(extract_price)


In [22]:
df_brazil_price.isna().sum()

price            0
price_cleaned    0
dtype: int64

In [23]:
df_brazil_price = df_brazil_price[df_brazil_price['price_cleaned'] != 1]


In [24]:
df_brazil_price = df_brazil_price[df_brazil_price['price_cleaned'] != 1]

In [25]:
df_brazil_price.drop(columns='price', inplace=True)

## Load New York prices.

In [26]:
import json
import pandas as pd
import os

#Open Sao Paulo json
path = '/workspaces/Coworking/src/results/New York/New_York_coworking_spaces.json'

with open(path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.json_normalize(data)

df_newyork_price= df[['price']]

In [27]:
import re

# Function to extract the price
def extract_price(text):
    if pd.isna(text):  # Handle missing values
        return None
    match = re.search(r'(\d+)', text)  # Extract the first number
    return int(match.group(1)) if match else None  # Convert to int if found

# Apply the function to the 'price' column
df_newyork_price['price_cleaned'] = df_newyork_price['price'].apply(extract_price)

# Display the results
print(df_newyork_price[['price', 'price_cleaned']])

                                               price  price_cleaned
0               Private Office\n from $ 1600\n/month         1600.0
1                Private Office\n from $ 473\n/month          473.0
2               Private Office\n from $ 3875\n/month         3875.0
3               Private Office\n from $ 1350\n/month         1350.0
4                                            POPULAR            NaN
5                Private Office\n from $ 775\n/month          775.0
6               Private Office\n from $ 3200\n/month         3200.0
7                Private Office\n from $ 600\n/month          600.0
8                Private Office\n from $ 615\n/month          615.0
9  Coworking Space\nfrom \n$420\n/month\nGET QUOT...          420.0


/tmp/ipykernel_26605/2228938316.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_newyork_price['price_cleaned'] = df_newyork_price['price'].apply(extract_price)


In [28]:
df_newyork_price.drop(columns='price', inplace=True)

/tmp/ipykernel_26605/2059561520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_newyork_price.drop(columns='price', inplace=True)


In [29]:
df_newyork_price.isna().sum()

price_cleaned    1
dtype: int64

In [30]:
df_newyork_price.dropna()

,price_cleaned
0,1600.0
1,473.0
2,3875.0
3,1350.0
5,775.0
6,3200.0
7,600.0
8,615.0
9,420.0


## Merge all the prices.

In [31]:
import pandas as pd

df_prices = pd.concat([df_euro, df_tokyo_price, df_brazil_price, df_newyork_price]).stack().reset_index(drop=True).to_frame(name='price')

df_prices.head()

,price
0,530.3760
1,204.5736
2,340.9560
3,1060.7520
4,243.5400


In [32]:
df_prices.shape

(223, 1)

In [33]:
df_prices.tail()

,price
218,775.0
219,3200.0
220,600.0
221,615.0
222,420.0
